In [1]:
# %run mom_data.ipynb
# %run cpi.ipynb
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import sqlalchemy
import investpy
# !pip install investpy
# !pip install yfinance
# import yfinance as yf
# import matplotlib.pyplot as plt
# %run explore_data.ipynb

## Helper Function

In [2]:
"""
Only use this roc function after getting the Year over Year 
percentage change because this function does not calculate the  
log difference or np.log().diff().
"""
def roc(df, n):
    data = df.copy()
    df1 = (data - data.shift(n))
    return df1


def RoC(df, n):
    """Calulates the First Difference of the 2nd Derivative (Rate of Change/Acceleration)"""    
    df1 = df.copy()
    df1 = (np.log(df1).diff(n))
    df1 = df1 - (df1.shift(1))
    df1 = df1 * 100
    return df1


def rateofchange(df, n):
    df1 = df.copy()
    df1 = ((df1 / df1.shift(n)) - 1 )* 100
    return df1
#     df1 = (df1 - 1) * 100

## Import Real GDP & Headline CPI from St.Louis FED (FRED)

In [3]:
data_source = 'fred'

start = datetime.datetime (1960, 1, 1) 
end = datetime.datetime (2030, 12, 1)

#  GDP Seasonally Adjusted annual rate
gdp = pdr.DataReader('GDPC1', data_source, start, end)
gdp.columns = ['gdp']
gdp = gdp.round(2)

#  Headline CPI Seasonally Adjusted
head_cpi = pdr.DataReader('CPIAUCSL', data_source, start, end)
# CPILFESL
head_cpi.columns = ['head_cpi']
head_cpi = head_cpi.round(2)

In [4]:
# gdp.tail()

In [5]:
# head_cpi['cpi_mom%'] = head_cpi['head_cpi'].pct_change(12) * 100 
# head_cpi.tail()

In [6]:
regime = pd.merge(gdp, head_cpi, left_index=True, right_index=True)
regime['gdp_mom%'] = regime['gdp'].pct_change() 
regime['gdp_yoy%'] = regime['gdp'].pct_change(4) * 100
regime['gdp_yoy%'] = regime['gdp_yoy%'].round(2)
# regime['gdp_roc1'] = (regime['gdp_yoy%']/regime['gdp_yoy%'].shift(4)) * 100
regime['gdp_roc']= RoC(regime['gdp'], 4)
# regime['gdp_roc1']= regime['gdp_roc'] + (regime['gdp_roc'].shift(8)*.001)
# regime['gdp_roc2']= rateofchange(regime['gdp'], 4)

regime['cpi_yoy%'] = regime['head_cpi'].pct_change(4) * 100
regime['cpi_yoy%'] = regime['cpi_yoy%'].round(2)
regime['cpi_roc'] = RoC(regime['head_cpi'], 4)
regime['cpi_roc1']= regime['cpi_roc'] - (regime['cpi_roc'].shift(8)/100)
# regime['cpi_roc2'] = rateofchange(regime['head_cpi'], 4)

In [7]:
# regime.dropna(inplace=True)
# print(regime.shape)
# regime.tail(16)

In [8]:
# regime.tail()

In [10]:
#  WTI Crude Monthly
crude = investpy.commodities.get_commodity_historical_data('Crude Oil WTI', 
                                                   from_date = '01/01/2000', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')

# Copper Monthly
copper = investpy.commodities.get_commodity_historical_data('Copper', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country='united states', 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')

# investpy.commodities.get_commodities_list()
us10yr = investpy.bonds.get_bond_historical_data('U.S. 10Y', 
                                        from_date = '01/01/1980', 
                                        to_date = '01/01/2030', 
#                                         as_json=False, 
                                        order='ascending', 
                                        interval='Monthly')

# Bitcoin Monthly
bitcoin = investpy.get_crypto_historical_data(
    crypto='bitcoin', 
    from_date='01/01/2012',
    to_date='12/01/2030',
    order='ascending',
    interval='Monthly')

# Bitcoin Monthly
litecoin = investpy.get_crypto_historical_data(
    crypto='litecoin', 
    from_date='01/01/2012',
    to_date='12/01/2030',
    order='ascending',
    interval='Monthly')

# US Dollar Index
dxy1 = investpy.indices.get_index_historical_data('US Dollar Index', 
                                        from_date = '01/01/1980', 
                                        to_date = '01/01/2030', 
                                        country='united states', 
                                        order='ascending', 
                                        interval='Monthly')

# TRCCRB (CRB - commodity index)
crb = investpy.indices.get_index_historical_data('TR/CC CRB', 
                                                 from_date = '01/01/1980', 
                                                 to_date = '01/01/2030', 
                                                 country='world', 
                                                 order='ascending', 
                                                 interval='Monthly')

# Gold
gold = investpy.commodities.get_commodity_historical_data('Gold', 
                                                   from_date = '01/01/1980', 
                                                   to_date = '01/01/2030', 
                                                   country=None, 
#                                                    as_json=False, 
                                                   order='ascending', 
                                                   interval='Monthly')


energy = investpy.get_etf_historical_data(
    etf='Energy Select Sector SPDR',
    country='United States',
    from_date='01/01/2012',
    to_date='12/01/2030', 
    interval='Monthly')



#  Dropping columns from Crude

crude.drop(['Open', 'High', 'Low', 'Volume', 'Currency'], axis=1, inplace=True)
copper.drop(['Open', 'High', 'Low', 'Volume', 'Currency'], axis=1, inplace=True)
us10yr.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
bitcoin.drop(['Open', 'High', 'Low', 'Volume', 'Currency'], axis=1, inplace=True)
dxy = dxy1.drop(['Open', 'High', 'Low', 'Currency', 'Volume'], axis=1)
crb.drop(['Open', 'High', 'Low', 'Currency', 'Volume'], axis=1, inplace=True)
gold.drop(['Open', 'High', 'Low', 'Volume', 'Currency'], axis=1, inplace=True)
energy.drop(['Open', 'High', 'Low', 'Volume', 'Currency', 'Exchange'], axis=1, inplace=True)
litecoin.drop(['Open', 'High', 'Low', 'Volume', 'Currency'], axis=1, inplace=True)
"""
TO DO: If returns are correlated to GIM (Growth/Inflation Model) then look further in number of highers on the 
Daily for the same commodity during a particular time period.


TO DO:  Also, look at the volume change (either percentage change, higher high, roc terms)
"""
crude.columns = ['crude']
copper.columns = ['copper']
us10yr.columns = ['us10yr']
litecoin.columns = ['ltc']
bitcoin.columns = ['btc']
dxy.columns = ['dxy']
crb.columns = ['crb']
gold.columns = ['gold']
energy.columns = ['energy']


# dxy.tail()
# crude.tail()
# copper.tail()
# bitcoin.tail()
# crb.tail()
# gold.tail()
# energy.tail()
# litecoin.head()

,crude
Date,
2020-12-01,48.52
2021-01-01,52.20
2021-02-01,61.50
2021-03-01,59.16
2021-04-01,63.48


In [10]:
# Merging gdp, head_cpi, crude, copper, us10yr, Bitcoin DataFrames (monthly)
df = pd.merge(gdp, head_cpi, left_index=True, right_index=True)
# print(df)
df['cpi_yoy%'] = df['head_cpi'].pct_change(4) * 100

df = pd.merge(df, crude, left_index=True, right_index=True)
df = pd.merge(df, copper, left_index=True, right_index=True)
df = pd.merge(df, us10yr, left_index=True, right_index=True)

# Renaming the DataFrame index so that merging is possible
df.index.rename(name='Date', inplace=True)

df = pd.merge(df, bitcoin, left_index=True, right_index=True)
df = pd.merge(df, dxy, left_index=True, right_index=True)
df = pd.merge(df, crb, left_index=True, right_index=True)
df = pd.merge(df, gold, left_index=True, right_index=True)
df = pd.merge(df, energy, left_index=True, right_index=True)
# df = pd.merge(df, litecoin, left_index=True, right_index=True)

df = df.round(4)
# df.tail(6)

In [11]:
def roc(df, n):
    """Calulates the First Difference of the 2nd Derivative (Rate of Change/Acceleration)"""    
    df1 = df.copy()
    df1 = (np.log(df1).diff(n))
    df1 = df1 - (df1.shift(1))
    df1 = df1 * 100
    return df1

In [12]:
# df['gdp_roc'] = df['gdp_yoy%'].diff().round(2) * 100
# df['cpi_roc'] = df['cpi_yoy%'].diff().round(2) * 100

df['gdp_roc'] = roc(df['gdp'], 3) * 100
df['cpi_roc'] = roc(df['head_cpi'], 3) * 100
df['crude_%chg'] = df['crude'].pct_change()*100
df['copper_%chg'] = df['copper'].pct_change()*100
df['us10yr_%chg'] = df['us10yr'].pct_change()*100
df['btc_%chg'] = df['btc'].pct_change()*100
# df['ltc_%chg'] = df['ltc'].pct_change()*100
df['dxy_%chg'] = df['dxy'].pct_change()*100
df['dxy_roc'] = roc(df['dxy'], 3)
df['crb_%chg'] = df['crb'].pct_change()*100
df['gold_%chg'] = df['gold'].pct_change()*100
df['ener_%chg'] = df['energy'].pct_change()*100
df['btc_roc'] = roc(df['btc'], 3) 

In [13]:
# df['GDP_roc_q2q'] = (np.log(df['GDP']).diff(4))   #  log diff/rate of change one quarter to same quarter of previous year
# df['GDP_roc_q2q'] = df['GDP_roc_q2q'] * 100  
# df['GDP_roc_q2q'] = (df['GDP_roc_q2q']) - (df['GDP_roc_q2q'].shift(1))    #  `Rate of Change` and the `first difference` * 100 == BPS
# df['GDP_roc_q2q'] = df['GDP_roc_q2q'] * 100

conditions = [
    (df['gdp_roc'] >= 0.0) & (df['cpi_roc'] < 0.0),
    (df['gdp_roc'] >= 0.0) & (df['cpi_roc'] > 0.0),
    (df['gdp_roc'] <= 0.0) & (df['cpi_roc'] > 0.0),
    (df['gdp_roc'] <= 0.0) & (df['cpi_roc'] < 0.0)
    ]

values = ['Regime 1', 'Regime 2', 'Regime 3', 'Regime 4']
df['Regimes'] = np.select(conditions, values)
df = df.round(2)
# df = df.dropna()
# if __name__ =="__main__":
    # print(df.tail(4))
#     print(df.shape)
#     print(df.tail(20))

In [14]:
# df.tail(1)

In [15]:
# df_mom['spr_10fedrate'] = df_mom['FedFunds'] - df_mom['10yr_yoy%']
# df.drop(['gdp', 'head_cpi',     # 'gdp_yoy%', 
#          'cpi_yoy%', 'crude', 'copper', 
#          'us10yr', 'gdp_roc', 'cpi_roc', 
#          'btc', 'dxy', 'crb', 'gold', 'energy'], axis=1, inplace=True)     

In [16]:
df1 = df.copy()

In [17]:
# df_mom['spr_10fedrate'].head()
# df['Regimes_shift_down'] = df['Regimes'].shift(-1)


In [18]:
df_regime2 = df1.copy()
# df_regime2['Regimes_shift_down'] = df['Regimes'].shift(-1)
# dfl = pd.merge(df1, bitcoin_df, left_index=True, right_index=True)
# df_regime2.drop(['dxy_%chg', 'gold_%chg'], axis=1, inplace=True)   
# df_regime2.drop(['Regimes'], axis=1, inplace=True)  

In [19]:
df_regime2.tail(40)

,gdp,head_cpi,cpi_yoy%,crude,copper,us10yr,btc,dxy,crb,gold,...,copper_%chg,us10yr_%chg,btc_%chg,dxy_%chg,dxy_roc,crb_%chg,gold_%chg,ener_%chg,btc_roc,Regimes
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,16129.42,227.84,3.01,98.48,3.79,1.80,5.5,79.29,312.31,1737.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2012-04-01,16198.81,229.19,2.28,104.87,3.83,1.92,4.9,78.78,305.95,1663.4,...,1.19,6.91,-10.91,-0.64,NaN,-2.04,-4.28,0.77,NaN,0
2012-07-01,16220.67,228.59,1.42,88.06,3.42,1.47,9.4,82.64,299.51,1610.5,...,-10.77,-23.40,91.84,4.90,NaN,-2.10,-3.18,-2.23,NaN,0
2012-10-01,16239.14,231.64,2.16,86.24,3.53,1.69,11.2,79.92,295.85,1717.5,...,3.13,15.24,19.15,-3.29,NaN,-1.22,6.64,3.28,NaN,0
2013-01-01,16382.96,231.68,1.69,97.49,3.72,1.98,20.4,79.21,303.99,1660.6,...,5.59,17.18,82.14,-0.89,-0.25,2.75,-3.31,7.52,71.51,Regime 1
2013-04-01,16403.18,231.80,1.14,93.46,3.19,1.67,139.2,81.75,288.13,1472.2,...,-14.39,-15.72,582.35,3.21,-1.63,-5.22,-11.35,1.20,126.89,Regime 3
2013-07-01,16531.69,232.90,1.89,105.03,3.12,2.59,106.2,81.45,283.94,1312.4,...,-2.16,54.69,-23.71,-0.37,2.98,-1.45,-10.85,5.30,-44.58,Regime 1
2013-10-01,16663.65,233.67,0.88,96.38,3.30,2.55,211.2,80.19,277.86,1323.6,...,5.64,-1.39,98.87,-1.55,-0.67,-2.14,0.85,4.81,8.79,Regime 3
2014-01-01,16616.54,235.29,1.56,97.49,3.22,2.64,938.8,81.31,283.31,1240.1,...,-2.28,3.61,344.51,1.40,-1.77,1.96,-6.31,-3.48,-42.86,Regime 3


In [20]:
#  TODO: columns = ['Regime', inflation indicators] - Regime column first

In [21]:
#  TODO: min , max, std, 25%, etc describe()/ info()
regime_describe = df_regime2.describe()
regime_describe

,gdp,head_cpi,cpi_yoy%,crude,copper,us10yr,btc,dxy,crb,gold,...,crude_%chg,copper_%chg,us10yr_%chg,btc_%chg,dxy_%chg,dxy_roc,crb_%chg,gold_%chg,ener_%chg,btc_roc
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,...,36.000000,36.000000,36.000000,36.000000,36.000000,33.000000,36.000000,36.00000,36.000000,33.000000
mean,17717.688108,243.203514,1.584595,65.370541,2.944324,2.042703,4202.029730,90.935405,218.654054,1442.705405,...,1.685556,0.290556,1.124722,50.609167,0.428611,-0.293939,-1.146389,0.43500,-0.905278,1.919697
std,1004.380019,10.282840,0.802647,23.521874,0.455037,0.629403,6446.306036,7.435990,56.563241,214.026925,...,27.576990,9.876418,22.694661,116.030941,3.459533,5.089153,9.463563,7.43772,11.383772,66.891743
min,16129.420000,227.840000,-0.230000,18.840000,2.060000,0.530000,4.900000,78.780000,117.200000,1161.900000,...,-63.460000,-17.950000,-57.080000,-52.540000,-6.560000,-9.640000,-31.180000,-11.35000,-28.890000,-204.830000
25%,16841.470000,235.290000,1.170000,48.240000,2.680000,1.690000,235.800000,81.750000,181.010000,1295.600000,...,-8.607500,-6.160000,-12.060000,-9.400000,-1.280000,-3.060000,-4.010000,-4.61000,-5.620000,-42.860000
50%,17668.200000,240.100000,1.690000,58.580000,2.970000,2.040000,698.700000,94.040000,194.530000,1347.700000,...,-1.345000,-1.875000,-0.655000,20.795000,0.905000,-0.670000,-1.640000,-1.25500,-0.785000,9.630000
75%,18654.380000,252.440000,2.090000,88.060000,3.220000,2.550000,7729.400000,97.340000,283.310000,1610.500000,...,12.445000,5.602500,10.242500,64.875000,2.752500,2.920000,2.705000,5.33750,4.562500,51.960000
max,19253.960000,262.230000,3.010000,105.030000,3.830000,3.150000,33108.100000,99.610000,312.310000,2004.600000,...,113.750000,23.080000,63.980000,582.350000,9.070000,10.970000,22.600000,18.23000,36.910000,126.890000


In [22]:
df1.corr()

,gdp,head_cpi,cpi_yoy%,crude,copper,us10yr,btc,dxy,crb,gold,...,crude_%chg,copper_%chg,us10yr_%chg,btc_%chg,dxy_%chg,dxy_roc,crb_%chg,gold_%chg,ener_%chg,btc_roc
gdp,1.000000,0.915995,0.144422,-0.642587,-0.374799,0.021188,0.659036,0.755694,-0.818422,0.058299,...,0.206083,0.209017,-0.003983,-0.243371,-0.158528,-0.077220,0.228469,0.265639,-0.034601,0.084189
head_cpi,0.915995,1.000000,0.117092,-0.621867,-0.265429,-0.206152,0.793253,0.643955,-0.812068,0.317522,...,0.194261,0.239864,-0.053455,-0.189626,-0.207106,-0.086847,0.182394,0.265462,-0.116040,0.029466
cpi_yoy%,0.144422,0.117092,1.000000,0.391661,0.431182,0.424365,0.094779,-0.204639,0.198404,0.086393,...,0.164531,0.134433,0.252658,-0.062785,-0.136980,0.200220,0.275914,-0.172387,0.231662,-0.279265
crude,-0.642587,-0.621867,0.391661,1.000000,0.747162,0.421367,-0.358274,-0.901415,0.940592,-0.064480,...,0.062439,-0.082821,0.224880,0.266159,0.038769,0.145994,0.171032,-0.390397,0.371070,-0.170206
copper,-0.374799,-0.265429,0.431182,0.747162,1.000000,0.108440,0.124442,-0.788902,0.659674,0.422501,...,0.260186,0.268709,0.315657,0.264210,-0.193915,0.011897,0.369993,-0.273605,0.436684,0.028678
us10yr,0.021188,-0.206152,0.424365,0.421367,0.108440,1.000000,-0.328130,-0.116201,0.296473,-0.740291,...,-0.117046,-0.119835,0.215751,-0.077211,0.102248,0.207232,-0.000335,-0.308496,0.184164,-0.346605
btc,0.659036,0.793253,0.094779,-0.358274,0.124442,-0.328130,1.000000,0.272837,-0.516538,0.527278,...,0.312837,0.287182,0.107237,0.010472,-0.268577,-0.138301,0.376191,0.124753,0.253722,0.179953
dxy,0.755694,0.643955,-0.204639,-0.901415,-0.788902,-0.116201,0.272837,1.000000,-0.908269,-0.181048,...,-0.102413,-0.010928,-0.183321,-0.305526,0.121374,0.014309,-0.198012,0.244450,-0.363399,0.092965
crb,-0.818422,-0.812068,0.198404,0.940592,0.659674,0.296473,-0.516538,-0.908269,1.000000,-0.097446,...,-0.052511,-0.159473,0.158932,0.244492,0.132446,0.151240,0.054240,-0.353650,0.308198,-0.139915
gold,0.058299,0.317522,0.086393,-0.064480,0.422501,-0.740291,0.527278,-0.181048,-0.097446,1.000000,...,0.336756,0.302965,0.011295,0.047534,-0.330909,-0.149919,0.289641,0.293723,0.028863,0.288919


In [23]:
def save_as_sqlite_database(df, sql_uri, tableName, index=True, index_label=None, verbose=True):
    """
    Saves DataFrame as a SQLite Databse
    
    Arguments
    ---------
    df: Pandas DataFrame
    filename: Filepath or name
    
    Returns
    -------
    db.sqlite3
    """
    
    try:
        if not sql_uri.endswith('.sqlite3'):
            sql_uri =+ '.sqlite3'
        if not sql_uri.startswith("sqlite:///"):
            raise ValueError("Database URI should start with 'sqlite:///'")
            
            
        from sqlalchemy import create_engine
        engine = create_engine(sql_uri)
        print(f"DataFrame saved @:{sql_uri[10:]}")
        df.to_sql(tableName, con=engine, index=index, index_label=index_label, if_exists='replace')
        
        if verbose:
            sql = f"SELECT * FROM {tableName} LIMIT 10;"
            print(pd.read_sql(sql, con=engine))
    except Exception as e:
            print("DataFrame could not be saved: ", sys.exc_info()[0])
            raise